In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
DATA_PATH = "/content/drive/MyDrive/2023/kaggle/AMP®-Parkinson's Disease Progression Prediction/"
supplemental = pd.read_csv(DATA_PATH +"supplemental_clinical_data.csv")
clinical = pd.read_csv(DATA_PATH +"train_clinical_data.csv")
peptides = pd.read_csv(DATA_PATH + "train_peptides.csv")
proteins = pd.read_csv(DATA_PATH + "train_proteins.csv")

In [ ]:
import itertools
import os
import pickle
import warnings

import argparse
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
import seaborn as sns

warnings.filterwarnings('ignore')

In [ ]:
target = pd.concat([supplemental, clinical], axis=0).reset_index(drop=True)
target = target[target.visit_month !=5].copy()

In [ ]:
target = target.rename(columns={'upd23b_clinical_state_on_medication':'medication'})

In [ ]:
clinical = clinical.rename(columns={'upd23b_clinical_state_on_medication':'medication'})

In [ ]:
for updrs_part in ['1', '2', '3', '4']:
    fig=px.histogram(target,
                     x = f'updrs_{updrs_part}',
                     color = 'medication',
                     width=800, height=500)
    fig.update_layout(template="plotly_dark")
    fig.update_layout(title_font_size=20)
    fig.show()

# UPDRS Score - Visit Month

In [ ]:
target['medication'] = target['medication'].fillna('Null')
target.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,medication
0,35_0,35,0,5.0,3.0,16.0,0.0,Null
1,35_36,35,36,6.0,4.0,20.0,0.0,Null
2,75_0,75,0,4.0,6.0,26.0,0.0,Null
3,75_36,75,36,1.0,8.0,38.0,0.0,On
4,155_0,155,0,NaN,NaN,0.0,NaN,Null


In [ ]:
parts = ['1','2','3','4']
fig = make_subplots(rows=4, cols=2, horizontal_spacing=0.10,
                    vertical_spacing=0.06,
                    column_titles=['Medication: "On"', 'Medication: "Off" or "Null"'])

for i, medication in enumerate([['On'], ['Off', 'Null']]):
  target_df = target.query('medication in @medication')
  for j, part in enumerate(parts):
    fig.add_trace(go.Box(x=target_df['visit_month'],
                         y=target_df[f'updrs_{part}']),
                  row=j+1, col=i+1)
    
    fig.update_xaxes(title_text='Visit Month', row=j+1, col=i+1)
    fig.update_yaxes(title_text=f'UPDRS Part {part}', row=j+1, col=1)

fig.update_layout(template='plotly_dark',
                  width=850, height=2000,
                  title_text ='<b>UPDRS Score - Visit Month',
                  showlegend =False)
fig.show()

# Peptide

In [ ]:
# 가장 아미노산 빈도가 높은 펩타이드 서열 구하기
peptides_agg = peptides[['patient_id','Peptide','PeptideAbundance']]
peptides_agg = peptides_agg.groupby(['patient_id', 'Peptide'])['PeptideAbundance'].aggregate(['mean','std'])
peptides_agg['CV_PeptideAbundance[%]'] = peptides_agg['std']/peptides_agg['mean']*100
#print(peptides_agg)
abundance_cv_mean = peptides_agg.groupby('Peptide')['CV_PeptideAbundance[%]'].mean().reset_index()
abundance_cv_mean = abundance_cv_mean.sort_values(
    by='CV_PeptideAbundance[%]', ascending=False).reset_index(drop=True)
peptide_cv_top5 = abundance_cv_mean[:10]['Peptide']
peptide_cv_top5
# peptide_agg_top5 = peptides_agg.query('Peptide in @peptide_cv_top5').reset_index()
# #peptide_agg_top5

# peptide_agg_top5['order']=0
# for i, peptide in enumerate(peptide_cv_top5):
#   index = peptide_agg_top5.query(f"Peptide=='{peptide}'").index
#   peptide_agg_top5.loc[index, 'order']=i
#   peptide_agg_top5.sort_values(by='order', inplace=True)

0                  SC(UniMod_4)SPELQQK
1             LPPTSAHGNVAEGETKPDPDVTER
2              ADDKETC(UniMod_4)FAEEGK
3                    TTPPVLDSDGSFFLYSK
4                    LFDSDPITVTVPVEVSR
5    ILTC(UniMod_4)M(UniMod_35)QGMEEIR
6              ALGISPFHEHAEVVFTANDSGPR
7           FNKPFVFLM(UniMod_35)IEQNTK
8                      KFPSGTFEQVSQLVK
9         VYC(UniMod_4)DMNTENGGWTVIQNR
Name: Peptide, dtype: object

In [ ]:
# 평균값에 대한 표준편차의 백분율
## 값이 높을수록 Peptide의 Abundance 값이 변동이 크다
fig = px.violin(peptide_agg_top5, y='Peptide', x='CV_PeptideAbundance[%]',
                color='Peptide', box=True,
                title='<b>Coeffcient of Variation (Top 5) of PeptideAbundance per patient_id',
                width=900, height=800)
fig.update_layout(template='plotly_dark', width=800, height=500,
                  showlegend=False,
                  xaxis=dict(title='Coffcient of variation [%] for PeptideAbundance per patient_id'),
                  yaxis=dict(title='<b>Peptide'))

fig.show()

In [ ]:
# peptide 순위와 약복용 여부에 따른 방문월 수
peptide_medication = pd.merge(peptides, 
                              target[['visit_id','medication']],
                              on='visit_id',)
peptide_medication['medication']=peptide_medication['medication'].fillna('Null')

peptide_cv_top5 = abundance_cv_mean[:5]['Peptide'].reset_index(drop=True)
tmp = peptide_medication[['visit_month','patient_id',
                          'Peptide','PeptideAbundance',
                          'medication']]
top5_peptide=tmp.query('Peptide in @peptide_cv_top5')

#top5_peptide
num_patients =50
fig = make_subplots(rows=5, cols=2,
                    horizontal_spacing=0.10,
                    vertical_spacing=0.06,
                    column_titles = ['<b>Medication:On', '<b>Medication:Off or Null'],
                    shared_yaxes=True) 
for i, medication in enumerate([['On'],['Off','Null']]):
  top5_peptide_df = top5_peptide.query('medication in @medication')
  for j, peptide in enumerate(peptide_cv_top5):
    peptide_df = top5_peptide_df.query(f'Peptide=="{peptide}"')
    fig.add_trace(go.Box(x=peptide_df['visit_month'],
                         y=peptide_df['PeptideAbundance']),
                  row=j+1, col=i+1)
    
    fig.update_xaxes(title_text='Visit Month', row=j+1, col=i+1)
    fig.update_yaxes(title_text=f'{peptide} Abundance', row=j+1, col=1)

fig.update_layout(template='plotly_dark',
                  width=800, height=2000,
                  title_text='<b>Peptide Abundance - Visit Month (Highest top5 CV)',
                  showlegend =False)

fig.show()                            

# Protein

In [ ]:
# NPX의 변동성이 큰 단백실 서열 5개 구하기
## CV가 클수록 데이터의 변동성이 높다.
proteins_agg = proteins[['patient_id','UniProt','NPX']]
proteins_agg = proteins_agg.groupby(['patient_id','UniProt'])['NPX'].aggregate(['mean','std'])
proteins_agg['CV_NPX[%]'] = proteins_agg['std'] / proteins_agg['mean']*100
NPX_cv_mean = proteins_agg.groupby('UniProt')['CV_NPX[%]'].mean().reset_index()
NPX_cv_mean = NPX_cv_mean.sort_values(by='CV_NPX[%]', ascending=False).reset_index(drop=True)
#NPX_cv_mean
protein_cv_top5 = NPX_cv_mean[:10]['UniProt']
protein_cv_top5

# protein_agg_top5 = proteins_agg.query('UniProt in @protein_cv_top5').reset_index()
# for i, protein in enumerate(protein_cv_top5):
#   index = protein_agg_top5.query(f'UniProt=="{protein}"').index
#   protein_agg_top5.loc[index, 'order'] =i
#   protein_agg_top5.sort_values(by='order', inplace=True)

0    P16152
1    P98160
2    Q8IWV7
3    P30086
4    P01861
5    Q9HDC9
6    P61278
7    P19827
8    P27169
9    P14314
Name: UniProt, dtype: object

In [ ]:
fig = px.violin(protein_agg_top5, y='UniProt', x='CV_NPX[%]', color='UniProt',
                box=True, title='<b>Coeffcient of Variation for NPX (Top 5)',
                width=900, height =500)
fig.update_layout(template ='plotly_dark',
                  showlegend=False,
                  xaxis=dict(title='Coeffcient of Variation [%] of NPX per patient_id'))
fig.show()

In [ ]:
# protein 순위와 약복용 여부에 따른 방문월 수
protein_medication = pd.merge(proteins, target[['visit_id','medication']], 
                              on='visit_id',)
protein_medication['medication']=protein_medication['medication'].fillna('Null')
protein_cv_top5 = NPX_cv_mean[:5]['UniProt'].reset_index(drop=True)
#protein_cv_top5
tmp = protein_medication[['visit_month','patient_id','UniProt',
                          'NPX','medication']]
top5_protein = tmp.query('UniProt in @protein_cv_top5')
fig = make_subplots(rows=5, cols=2,
                    horizontal_spacing=0.10,
                    vertical_spacing=0.06,
                    column_titles=['<b>Medication: On','<b>Medication: Off of Null'],
                    shared_yaxes=True)
for i, medication in enumerate([['On'],['Off', 'Null']]):
  top5_protein_df = top5_protein.query('medication in @medication')
  for j, protein in enumerate(protein_cv_top5):
    protein_df = top5_protein_df.query(f'UniProt=="{protein}"')
    fig.add_trace(go.Box(x=protein_df['visit_month'], y=protein_df['NPX']),
                  row=j+1, col=i+1)
    fig.update_xaxes(title_text='Visit Month', row=j+1, col=i+1)
    fig.update_yaxes(title_text=f'{protein} NPX', row=j+1, col=1)

fig.update_layout(template ='plotly_dark',
                  width=800, height=2000,
                  title_text ='<b>NPX - Visit Month (Highset top5 CV)',
                  showlegend =False)

fig.show() 


# Preprocessing

In [ ]:
# peptideabundance에 대한 변동 계수
def get_peptide_cadidates(peptides, num_candidates):
  train_peptides_df_agg = peptides[['patient_id', 'Peptide','PeptideAbundance']]
  train_peptides_df_agg = train_peptides_df_agg.groupby(['patient_id','Peptide'])['PeptideAbundance'].aggregate(['mean','std'])
  train_peptides_df_agg['CV_PeptideAbundance[%]'] = train_peptides_df_agg['std']/train_peptides_df_agg['mean']*100

  abundance_cv_mean = train_peptides_df_agg.groupby('Peptide')['CV_PeptideAbundance[%]'].mean().reset_index()
  abundance_cv_mean = abundance_cv_mean.sort_values(
      by='CV_PeptideAbundance[%]', ascending=False).reset_index(drop=True)

  peptide_candidates = abundance_cv_mean.loc[:num_candidates-1, 'Peptide']
  return peptide_candidates


# protein NPX에 대한 변동 계수
def get_protein_cadidates(proteins, num_candidates):
  train_proteins_df_agg = proteins[['patient_id', 'UniProt', 'NPX']]
  train_proteins_df_agg = train_proteins_df_agg.groupby(['patient_id', 'UniProt'])['NPX'].aggregate(['mean', 'std'])
  train_proteins_df_agg['CV_NPX[%]'] = train_proteins_df_agg['std']/train_proteins_df_agg['mean']*100

  NPX_cv_mean = train_proteins_df_agg.groupby('UniProt')['CV_NPX[%]'].mean().reset_index()
  NPX_cv_mean = NPX_cv_mean.sort_values(
      by='CV_NPX[%]', ascending=False).reset_index(drop=True)

  protein_candidates = NPX_cv_mean.loc[:num_candidates-1, 'UniProt']
  return protein_candidates

In [ ]:
def preprocessing_train(clinical, peptides, proteins,
                        peptide_candidates, protein_candidates):
  
  def create_peptide_candidate(peptides, peptide_candidates):
    peptide_candidate_df = peptides.query(f'Peptide in @peptide_candidates')
    visit_ids = clinical['visit_id'].unique()
    peptide_dict_list = []
    for visit_id in visit_ids:
      peptides_df = peptide_candidate_df.query(f'visit_id=="{visit_id}"')
      peptides_df_ = peptides_df[['Peptide']].values
      PeptideAbundances = peptides_df['PeptideAbundance'].values
      peptide_dict = dict(zip(peptide_candidates, [np.nan]*len(peptide_candidates)))
      for peptide, PeptideAbundance in zip(peptides_df_['Peptide'], PeptideAbundances):
          peptide_dict[peptides] = PeptideAbundance
      peptide_dict['visit_id'] = visit_id
      peptide_dict_list.append(peptide_dict)

    return pd.DataFrame(peptide_dict_list)


  def create_protein_candidate(proteins, protein_candidates):
    protein_candidate_df = proteins.query(f'UniProt in @protein_candidates')
    visit_ids = clinical['visit_id'].unique()
    protein_dict_list = []
    for visit_id in visit_ids:
      proteins_df = protein_candidate_df.query(f'visit_id=="{visit_id}"')
      UniProts = proteins_df['UniProt'].values
      NPXs = proteins_df['NPX'].values
      protein_dict = dict(zip(protein_candidates, [np.nan]*len(protein_candidates)))
      for UniProt, NPX in zip(UniProts, NPXs):
          protein_dict[UniProt] = NPX
      protein_dict['visit_id'] = visit_id
      protein_dict_list.append(protein_dict)

    return pd.DataFrame(protein_dict_list)


  df = clinical.copy()
  peptide_candidate_df = create_peptide_candidate(peptides, peptide_candidates)
  protein_candidate_df = create_protein_candidate(proteins, protein_candidates)
  df = pd.merge(df, peptide_candidate_df, on='visit_id', how='left')  
  df = pd.merge(df, protein_candidate_df, on='visit_id', how='left')
  if 'medication' in df.columns:
      df.drop(['medication'], axis=1, inplace=True)

  return df

In [ ]:
X_dict, y_dict = create_X_y_dict(train_df)

  0%|          | 0/4 [00:00<?, ?it/s]

e.g. updrs1_plus_month6

X:


,visit_id,visit_month,SC(UniMod_4)SPELQQK,LPPTSAHGNVAEGETKPDPDVTER,ADDKETC(UniMod_4)FAEEGK,TTPPVLDSDGSFFLYSK,LFDSDPITVTVPVEVSR,ILTC(UniMod_4)M(UniMod_35)QGMEEIR,ALGISPFHEHAEVVFTANDSGPR,FNKPFVFLM(UniMod_35)IEQNTK,KFPSGTFEQVSQLVK,VYC(UniMod_4)DMNTENGGWTVIQNR,P16152,P98160,Q8IWV7,P30086,P01861,Q9HDC9,P61278,P19827,P27169,P14314
0,673_0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,673_6,6,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,755_0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,755_6,6,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,755_12,12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


y:


,visit_id,updrs_1
0,673_6,3.000000
1,673_12,6.000000
2,755_6,1.000000
3,755_12,6.000000
4,755_18,1.000000


In [ ]:
print(X_dict.keys())
print(y_dict.keys())

dict_keys(['updrs1_plus_month0', 'updrs1_plus_month6', 'updrs1_plus_month12', 'updrs1_plus_month24', 'updrs2_plus_month0', 'updrs2_plus_month6', 'updrs2_plus_month12', 'updrs2_plus_month24', 'updrs3_plus_month0', 'updrs3_plus_month6', 'updrs3_plus_month12', 'updrs3_plus_month24', 'updrs4_plus_month0', 'updrs4_plus_month6', 'updrs4_plus_month12', 'updrs4_plus_month24'])
dict_keys(['updrs1_plus_month0', 'updrs1_plus_month6', 'updrs1_plus_month12', 'updrs1_plus_month24', 'updrs2_plus_month0', 'updrs2_plus_month6', 'updrs2_plus_month12', 'updrs2_plus_month24', 'updrs3_plus_month0', 'updrs3_plus_month6', 'updrs3_plus_month12', 'updrs3_plus_month24', 'updrs4_plus_month0', 'updrs4_plus_month6', 'updrs4_plus_month12', 'updrs4_plus_month24'])


# Modeling

복붙

In [ ]:
INPUT_DIR = "/content/drive/MyDrive/2023/kaggle/AMP®-Parkinson's Disease Progression Prediction"
train_clinical_df = pd.read_csv(os.path.join(INPUT_DIR, 'train_clinical_data.csv'))
train_peptides_df = pd.read_csv(os.path.join(INPUT_DIR, 'train_peptides.csv'))
train_proteins_df = pd.read_csv(os.path.join(INPUT_DIR, 'train_proteins.csv'))
supplemental_clinical_df = pd.read_csv(os.path.join(INPUT_DIR, 'supplemental_clinical_data.csv'))

In [ ]:
# Get peptide catidates from higher variability of abundance
def get_peptide_cadidates(peptides_df, num_candidates):
    # Calculate the coefficient of variation (CV) for PeptideAbundance per patient_ids and Peptides
    train_peptides_df_agg = peptides_df[['patient_id', 'Peptide', 'PeptideAbundance']]
    train_peptides_df_agg = train_peptides_df_agg\
                            .groupby(['patient_id', 'Peptide'])['PeptideAbundance']\
                            .aggregate(['mean', 'std'])
    train_peptides_df_agg['CV_PeptideAbundance[%]'] = \
        train_peptides_df_agg['std'] / train_peptides_df_agg['mean'] * 100

    # Mean CV value of Peptides
    abundance_cv_mean = train_peptides_df_agg\
                        .groupby('Peptide')['CV_PeptideAbundance[%]']\
                        .mean().reset_index()
    abundance_cv_mean = abundance_cv_mean.sort_values(
        by='CV_PeptideAbundance[%]', ascending=False).reset_index(drop=True)

    # Get peptide candidates
    peptide_candidates = abundance_cv_mean.loc[:num_candidates-1, 'Peptide']
    return peptide_candidates


# Get protein candidate from proteins with higher variability of NPX
def get_protein_cadidates(proteins_df, num_candidates):
    # Calculate the coefficient of variation (CV) for NPX per patient_ids and UniProt
    train_proteins_df_agg = train_proteins_df[['patient_id', 'UniProt', 'NPX']]
    train_proteins_df_agg = train_proteins_df_agg.groupby(['patient_id', 'UniProt'])['NPX']\
                            .aggregate(['mean', 'std'])
    train_proteins_df_agg['CV_NPX[%]'] = \
        train_proteins_df_agg['std'] / train_proteins_df_agg['mean'] * 100

    # Mean CV value for UniProt
    NPX_cv_mean = train_proteins_df_agg.groupby('UniProt')['CV_NPX[%]'].mean().reset_index()
    NPX_cv_mean = NPX_cv_mean.sort_values(by='CV_NPX[%]', ascending=False).reset_index(drop=True)

    # Get peptide candidates
    protein_candidates = NPX_cv_mean.loc[:num_candidates-1, 'UniProt']
    return protein_candidates
    


def preprocessing_train_data(clinical_df, peptides_df, proteins_df,
                       peptide_candidates, protein_candidates):
    
    
    # Create dataframe with columns:['visit_id', 'peptide_candidata1', 'peptide_candidata2', ...]
    def create_peptide_candidate_df(peptides_df, peptide_candidates):
        peptide_candidate_df = peptides_df.query(f'Peptide in @peptide_candidates')
        visit_ids = clinical_df['visit_id'].unique()
        peptide_dict_list = []
        for visit_id in visit_ids:
            peptides_df = peptide_candidate_df.query(f'visit_id=="{visit_id}"')
            peptides = peptides_df['Peptide'].values
            PeptideAbundances = peptides_df['PeptideAbundance'].values
            peptide_dict = dict(zip(peptide_candidates, [np.nan]*len(peptide_candidates)))
            for peptide, PeptideAbundance in zip(peptides, PeptideAbundances):
                peptide_dict[peptide] = PeptideAbundance
            peptide_dict['visit_id'] = visit_id
            peptide_dict_list.append(peptide_dict)

        return pd.DataFrame(peptide_dict_list)
    
    # Create dataframe with columns:['visit_id', 'protein_candidata1', 'protein_candidata2', ...]
    def create_protein_candidate_df(proteins_df, protein_candidates):
        protein_candidate_df = proteins_df.query(f'UniProt in @protein_candidates')
        visit_ids = clinical_df['visit_id'].unique()
        protein_dict_list = []
        for visit_id in visit_ids:
            proteins_df = protein_candidate_df.query(f'visit_id=="{visit_id}"')
            UniProts = proteins_df['UniProt'].values
            NPXs = proteins_df['NPX'].values
            protein_dict = dict(zip(protein_candidates, [np.nan]*len(protein_candidates)))
            for UniProt, NPX in zip(UniProts, NPXs):
                protein_dict[UniProt] = NPX
            protein_dict['visit_id'] = visit_id
            protein_dict_list.append(protein_dict)

        return pd.DataFrame(protein_dict_list)
  
    df = clinical_df.copy()
    peptide_candidate_df = create_peptide_candidate_df(train_peptides_df, peptide_candidates)
    protein_candidate_df = create_protein_candidate_df(train_proteins_df, protein_candidates)
    df = pd.merge(df, peptide_candidate_df, on='visit_id', how='left')
    df = pd.merge(df, protein_candidate_df, on='visit_id', how='left')
    if 'upd23b_clinical_state_on_medication' in df.columns:
        df.drop(['upd23b_clinical_state_on_medication'], axis=1, inplace=True)
#     df['upd23b_clinical_state_on_medication'] = \
#         df['upd23b_clinical_state_on_medication'].fillna('Off')
#     df = pd.get_dummies(df, columns=['upd23b_clinical_state_on_medication'])
    
    return df

In [ ]:
train_clinical_df = pd.read_csv(os.path.join(INPUT_DIR, 'train_clinical_data.csv'))
train_peptides_df = pd.read_csv(os.path.join(INPUT_DIR, 'train_peptides.csv'))
train_proteins_df = pd.read_csv(os.path.join(INPUT_DIR, 'train_proteins.csv'))
NUM_CANDIDATES = 10

peptide_candidates = get_peptide_cadidates(train_peptides_df, num_candidates=NUM_CANDIDATES)
protein_candidates = get_protein_cadidates(train_proteins_df, num_candidates=NUM_CANDIDATES)
train_df = preprocessing_train_data(train_clinical_df,
                                    train_peptides_df,
                                    train_proteins_df,
                                    peptide_candidates,
                                    protein_candidates)

print('train_df:')
display(train_df.head(10).style.set_properties(
    **{"background-color": "#212636","color":"white","border": "1.5px solid white"}))

train_df:


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,SC(UniMod_4)SPELQQK,LPPTSAHGNVAEGETKPDPDVTER,ADDKETC(UniMod_4)FAEEGK,TTPPVLDSDGSFFLYSK,LFDSDPITVTVPVEVSR,ILTC(UniMod_4)M(UniMod_35)QGMEEIR,ALGISPFHEHAEVVFTANDSGPR,FNKPFVFLM(UniMod_35)IEQNTK,KFPSGTFEQVSQLVK,VYC(UniMod_4)DMNTENGGWTVIQNR,P16152,P98160,Q8IWV7,P30086,P01861,Q9HDC9,P61278,P19827,P27169,P14314
0,55_0,55,0,10.000000,6.000000,15.000000,nan,47171.000000,129775.000000,70412.000000,80617.800000,130107.000000,57246.200000,1842000.000000,90349.000000,109357.000000,16627.100000,47171.000000,21970.100000,57246.200000,89747.300000,18430.100000,365475.000000,6323.720000,36232.900000,13998.000000,44112.600000
1,55_3,55,3,10.000000,7.000000,25.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,55_6,55,6,8.000000,10.000000,34.000000,nan,44669.000000,874897.000000,63052.400000,78095.300000,288345.000000,158574.000000,1680840.000000,91683.700000,134524.000000,13055.000000,44669.000000,20700.700000,158574.000000,72686.000000,31112.400000,405676.000000,6713.990000,14899.500000,12089.400000,45298.000000
3,55_9,55,9,8.000000,9.000000,30.000000,0.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,55_12,55,12,10.000000,10.000000,41.000000,0.000000,44159.500000,828847.000000,72325.000000,81248.800000,282297.000000,152944.000000,1565850.000000,107563.000000,108812.000000,18267.400000,44159.500000,19547.800000,152944.000000,86064.900000,22094.800000,303953.000000,5116.710000,nan,14222.700000,46491.100000
5,55_18,55,18,7.000000,13.000000,38.000000,0.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,55_24,55,24,16.000000,9.000000,49.000000,0.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,55_30,55,30,14.000000,13.000000,49.000000,0.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,55_36,55,36,17.000000,18.000000,51.000000,0.000000,48076.200000,960251.000000,537430.000000,6875.790000,71400.100000,177998.000000,1005250.000000,nan,64580.400000,15751.900000,48076.200000,21385.700000,177998.000000,90811.900000,21304.200000,303597.000000,5703.540000,nan,11837.100000,46482.600000
9,55_42,55,42,12.000000,20.000000,41.000000,0.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
def create_X_y_train_dataset(df, updrs_part, plus_month):
    # df: train_df, created above
    # updrs_part: 1 to 4
    # plus_month: 0, 6, 12, 24
    df_ = df.dropna(subset=[f'updrs_{updrs_part}'])
    X_visit_ids = []
    y_visit_ids = []
    patient_ids = df['patient_id'].unique()
    for i, patient_id in enumerate(patient_ids):
        patient_df = df_[df_['patient_id']==patient_id]
        plus_months = patient_df['visit_month'] + plus_month
        plus_months = patient_df.query('visit_month in @plus_months')['visit_month']
        original_months = plus_months - plus_month
        patient_id = str(patient_id)
        X_visit_id = [patient_id+'_'+str(original_month) for original_month in original_months]
        y_visit_id = [patient_id+'_'+str(plus_month) for plus_month in plus_months]
        X_visit_ids.extend(X_visit_id)
        y_visit_ids.extend(y_visit_id)
    
    X = df_.query('visit_id in @X_visit_ids')
    X = X.drop(['patient_id', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4'], axis=1)
    X.reset_index(drop=True, inplace=True)
    
    y = df_.query('visit_id in @y_visit_ids')
    y = y[['visit_id', f'updrs_{updrs_part}']]
    y.reset_index(drop=True, inplace=True)
    
    return X, y

def create_X_y_dict(df):
    X_dict = {}
    y_dict = {}
    for updrs_part in tqdm([1, 2, 3, 4]):
        for plus_month in [0, 6, 12, 24]:
            X, y = create_X_y_train_dataset(df, updrs_part, plus_month)
            key = f'updrs{updrs_part}_plus_month{plus_month}'
            X_dict[key] = X
            y_dict[key] = y
    return X_dict, y_dict

In [ ]:
X_dict, y_dict = create_X_y_dict(train_df)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# For example,
updrs = 2
plus_month = 6
key = f'updrs{updrs_part}_plus_month{plus_month}'

print(f'e.g. {key}\n')
X = X_dict[key].copy()
y = y_dict[key].copy()
print('X:')
display(X.head(5).style.set_properties(
    **{"background-color": "#212636","color":"white","border": "1.5px solid white"}))
print('y:')
display(y.head(5).style.set_properties(
    **{"background-color": "#212636","color":"white","border": "1.5px solid white"}))

e.g. updrs1_plus_month6

X:


,visit_id,visit_month,SC(UniMod_4)SPELQQK,LPPTSAHGNVAEGETKPDPDVTER,ADDKETC(UniMod_4)FAEEGK,TTPPVLDSDGSFFLYSK,LFDSDPITVTVPVEVSR,ILTC(UniMod_4)M(UniMod_35)QGMEEIR,ALGISPFHEHAEVVFTANDSGPR,FNKPFVFLM(UniMod_35)IEQNTK,KFPSGTFEQVSQLVK,VYC(UniMod_4)DMNTENGGWTVIQNR,P16152,P98160,Q8IWV7,P30086,P01861,Q9HDC9,P61278,P19827,P27169,P14314
0,55_0,0,47171.000000,129775.000000,70412.000000,80617.800000,130107.000000,57246.200000,1842000.000000,90349.000000,109357.000000,16627.100000,47171.000000,21970.100000,57246.200000,89747.300000,18430.100000,365475.000000,6323.720000,36232.900000,13998.000000,44112.600000
1,55_3,3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,55_6,6,44669.000000,874897.000000,63052.400000,78095.300000,288345.000000,158574.000000,1680840.000000,91683.700000,134524.000000,13055.000000,44669.000000,20700.700000,158574.000000,72686.000000,31112.400000,405676.000000,6713.990000,14899.500000,12089.400000,45298.000000
3,55_12,12,44159.500000,828847.000000,72325.000000,81248.800000,282297.000000,152944.000000,1565850.000000,107563.000000,108812.000000,18267.400000,44159.500000,19547.800000,152944.000000,86064.900000,22094.800000,303953.000000,5116.710000,nan,14222.700000,46491.100000
4,55_18,18,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


y:


,visit_id,updrs_1
0,55_6,8.000000
1,55_9,8.000000
2,55_12,10.000000
3,55_18,7.000000
4,55_24,16.000000


#Modeling

In [ ]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
import lightgbm as lgb
import optuna

# SMAPE Metric
def smape(y_true, y_pred):
    y_true = y_true + 1
    y_pred = y_pred + 1
    numerator = np.abs(y_true - y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    positive_index = (y_true!=0) | (y_pred!=0)
    smape = np.zeros(len(y_true))
    smape[positive_index] = numerator[positive_index] / denominator[positive_index]
    smape = 100 * np.mean(smape)
    return smape

class Callback:
    def __init__(self):
        self.models = {}
    
    def register_model(self, trial_number, model):
        self.models[str(trial_number)] = model
    
    def unregister_model(self, trial_number):
        self.models.pop(str(trial_number), None)
    
    def unregister_other_model(self, trial_number):
        model = self.models.pop(str(trial_number), None)
        self.models.clear()
        self.models[str(trial_number)] = model
    
    def get_model(self, trial_number):
        return self.models[str(trial_number)]
    
    def __call__(self, study, trial):
        if study.best_trial.number == trial.number:
            self.unregister_other_model(study.best_trial.number)
            self.save(study)
        else:
            self.unregister_model(trial.number)
            # save study.trials_dataframe()
    
    def save(self, study):
        model = self.get_model(study.best_trial.number)
        # save model
        # save study.best_params
        # save study.trials_dataframe()

class LGBObjective():
    
    def __init__(self, X_train, y_train, X_valid, y_valid, callback):
        self.X_train = X_train
        self.y_train = y_train
        self.X_valid = X_valid
        self.y_valid = y_valid
        self.callback = callback
    
    def __call__(self, trial):
        
        params = {
            'n_iter'           : 200,
            'verbosity'        : -1,
            'objective'        : trial.suggest_categorical('objective', ['l1', 'l2']),
            'random_state'     : 0,
            'extra_trees'      : True,
            'colsample_bytree' : 0.1, #trial.suggest_float('colsample_bytree', 0.1, 1.0),
            'colsample_bynode' : 0.1, #trial.suggest_float('colsample_bynode', 0.1, 1.0),
            'max_depth'        : 16, # trial.suggest_int('max_depth', 4, 32),
            'learning_rate'    : 0.01, #trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'lambda_l1'        : trial.suggest_float('lambda_l1', 1e-2, 10.0),
            'lambda_l2'        : trial.suggest_float('lambda_l2', 1e-2, 10.0),
            'num_leaves'       : trial.suggest_int('num_leaves', 8, 512),
        }        
        
        lgb_train = lgb.Dataset(self.X_train, self.y_train)
        model = lgb.train(params, lgb_train, verbose_eval=0)
        self.callback.register_model(trial.number, model)
        self.y_pred = model.predict(self.X_valid)
        score = smape(self.y_valid, self.y_pred)
        
        return score  

#Training

In [ ]:
args = argparse.Namespace(
    debug = False,
    n_folds = 5, # If debug is True, only one fold is calculated.
    n_trials = 10,
)

def add_fold_column(X, y, n_folds):
    X_ = X.copy()
    y_ = y.copy()
    X_['fold'] = -1
    y_['fold'] = -1
    num_bins = int(np.floor(1+np.log2(len(y))))
    y_bin = pd.cut(y_.iloc[:,1], # updrs_X score
                   bins=num_bins,
                   labels=False)
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X_, y_bin)):
        X_.loc[valid_idx, 'fold'] = fold
        y_.loc[valid_idx, 'fold'] = fold
    return X_, y_


best_params = {}
best_values = {}
best_models = {}
for updrs_part in [1, 2, 3, 4]:
    for plus_month in [0, 6, 12, 24]:
        key = f'updrs{updrs_part}_plus_month{plus_month}'
        X = X_dict[key].copy()
        y = y_dict[key].copy()
        # Add fold number column
        X, y = add_fold_column(X, y, n_folds=args.n_folds)
        n_folds = args.n_folds if not args.debug else 1
        score = 0
        for fold in tqdm(range(n_folds)):
            print(f"\n{'>'*15} Fold {fold} {'<'*15}")
            # Split to train and valid data
            train_idx = X['fold']!=fold
            valid_idx = X['fold']==fold
            X_train = X.loc[train_idx].drop(['visit_id', 'fold'], axis=1)
            X_valid = X.loc[valid_idx].drop(['visit_id', 'fold'], axis=1)
            y_train = y.loc[train_idx][f'updrs_{updrs_part}']
            y_valid = y.loc[valid_idx][f'updrs_{updrs_part}']
            # Train
            callback = Callback()
            lgb_objective = LGBObjective(X_train, 
                                         y_train,
                                         X_valid,
                                         y_valid,
                                         callback)
            study = optuna.create_study(
                direction='minimize',
                study_name=f'lgb_regression_updrs{updrs_part}_plus_month{plus_month}'
            )
            study.optimize(lgb_objective, n_trials=args.n_trials, show_progress_bar=True)
            key = f'updrs{updrs_part}_plus_month{plus_month}_fold{fold}'
            best_params[key] = study.best_params
            best_values[key] = study.best_value
            best_model = callback.get_model(study.best_trial.number)
            best_models[key] = best_model
            pickle.dump(best_model, open(f'{key}.pkl', 'wb'))

  0%|          | 0/5 [00:00<?, ?it/s]

[I 2023-05-08 03:35:39,308] A new study created in memory with name: lgb_regression_updrs1_plus_month0



>>>>>>>>>>>>>>> Fold 0 <<<<<<<<<<<<<<<


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-05-08 03:35:40,176] Trial 0 finished with value: 55.65458545784217 and parameters: {'objective': 'l1', 'lambda_l1': 4.563286328740253, 'lambda_l2': 4.971726384505094, 'num_leaves': 379}. Best is trial 0 with value: 55.65458545784217.
[I 2023-05-08 03:35:41,400] Trial 1 finished with value: 55.980630640593155 and parameters: {'objective': 'l2', 'lambda_l1': 1.0243596122586203, 'lambda_l2': 1.774619892058501, 'num_leaves': 325}. Best is trial 0 with value: 55.65458545784217.
[I 2023-05-08 03:35:42,064] Trial 2 finished with value: 55.57780107497924 and parameters: {'objective': 'l1', 'lambda_l1': 8.450377870489016, 'lambda_l2': 8.060180273534188, 'num_leaves': 312}. Best is trial 2 with value: 55.57780107497924.
[I 2023-05-08 03:35:42,411] Trial 3 finished with value: 55.601864491877336 and parameters: {'objective': 'l1', 'lambda_l1': 6.521306224858017, 'lambda_l2': 6.9339495576346515, 'num_leaves': 358}. Best is trial 2 with value: 55.57780107497924.
[I 2023-05-08 03:35:45,795] 

...

[I 2023-05-08 03:37:39,540] Trial 0 finished with value: 81.25614995722465 and parameters: {'objective': 'l2', 'lambda_l1': 1.0827967414394792, 'lambda_l2': 4.339592541643063, 'num_leaves': 455}. Best is trial 0 with value: 81.25614995722465.
[I 2023-05-08 03:37:39,628] Trial 1 finished with value: 74.87070222222974 and parameters: {'objective': 'l1', 'lambda_l1': 3.8950817462728646, 'lambda_l2': 4.8156497814604675, 'num_leaves': 49}. Best is trial 1 with value: 74.87070222222974.
[I 2023-05-08 03:37:39,714] Trial 2 finished with value: 81.16020174878292 and parameters: {'objective': 'l2', 'lambda_l1': 6.627087646906642, 'lambda_l2': 8.62747160457174, 'num_leaves': 422}. Best is trial 1 with value: 74.87070222222974.
[I 2023-05-08 03:37:39,801] Trial 3 finished with value: 81.09758047509575 and parameters: {'objective': 'l2', 'lambda_l1': 7.712711291992881, 'lambda_l2': 5.015361823670622, 'num_leaves': 27}. Best is trial 1 with value: 74.87070222222974.
[I 2023-05-08 03:37:39,868] Tria

[I 2023-05-08 03:37:40,362] A new study created in memory with name: lgb_regression_updrs4_plus_month24


[I 2023-05-08 03:37:40,338] Trial 9 finished with value: 81.17581422621646 and parameters: {'objective': 'l2', 'lambda_l1': 1.6630676554090125, 'lambda_l2': 6.528902247574183, 'num_leaves': 425}. Best is trial 8 with value: 74.81668396573998.

>>>>>>>>>>>>>>> Fold 4 <<<<<<<<<<<<<<<


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-05-08 03:37:40,456] Trial 0 finished with value: 73.20153149846541 and parameters: {'objective': 'l1', 'lambda_l1': 9.716163143517738, 'lambda_l2': 0.4868520196706277, 'num_leaves': 341}. Best is trial 0 with value: 73.20153149846541.
[I 2023-05-08 03:37:40,541] Trial 1 finished with value: 72.40615211391096 and parameters: {'objective': 'l1', 'lambda_l1': 6.02964421351363, 'lambda_l2': 0.8709544580091512, 'num_leaves': 350}. Best is trial 1 with value: 72.40615211391096.
[I 2023-05-08 03:37:40,603] Trial 2 finished with value: 79.06200209194256 and parameters: {'objective': 'l2', 'lambda_l1': 3.6866935778160332, 'lambda_l2': 5.131271439979052, 'num_leaves': 136}. Best is trial 1 with value: 72.40615211391096.
[I 2023-05-08 03:37:40,691] Trial 3 finished with value: 79.18226063012784 and parameters: {'objective': 'l2', 'lambda_l1': 1.3852634424882428, 'lambda_l2': 7.296143554229854, 'num_leaves': 386}. Best is trial 1 with value: 72.40615211391096.
[I 2023-05-08 03:37:40,754] T

In [ ]:
print('-'*50)
print(f'Model \t\t\t| SMAPE score')
print('-'*50)

n_folds = args.n_folds if not args.debug else 1
smape_total_mean = 0
for updrs_part in [1, 2, 3, 4]:
    for plus_month in [0, 6, 12, 24]:
        smape_fold_mean = 0
        key = f'updrs{updrs_part}_plus_month{plus_month}'
        for fold in range(n_folds):
            smape_fold = best_values[f'{key}_fold{fold}']
            smape_fold_mean += smape_fold / float(n_folds)
        print(f'updrs{updrs_part}_plus_month{plus_month}\t| {smape_fold_mean:.2f}')
        smape_total_mean += smape_fold_mean / 16.0

print('-'*50)
print(f'SMAPE mean\t\t| {smape_total_mean:.2f}')

--------------------------------------------------
Model 			| SMAPE score
--------------------------------------------------
updrs1_plus_month0	| 56.09
updrs1_plus_month6	| 53.35
updrs1_plus_month12	| 54.48
updrs1_plus_month24	| 54.31
updrs2_plus_month0	| 71.52
updrs2_plus_month6	| 61.37
updrs2_plus_month12	| 71.73
updrs2_plus_month24	| 72.36
updrs3_plus_month0	| 70.74
updrs3_plus_month6	| 55.44
updrs3_plus_month12	| 71.90
updrs3_plus_month24	| 72.63
updrs4_plus_month0	| 50.44
updrs4_plus_month6	| 49.62
updrs4_plus_month12	| 58.84
updrs4_plus_month24	| 73.18
--------------------------------------------------
SMAPE mean		| 62.38
